# 🌾 Train Intent Classifier - PhoBERT for Agricultural Chatbot

Notebook này dùng để train lại model Intent Classification với data mới.

**Model**: PhoBERT (vinai/phobert-base)  
**Task**: Multi-class Intent Classification  
**Labels**: knowledge_query, financial_query, device_control, sensor_query, unknown

## 📦 1. Cài đặt thư viện

In [ ]:
!pip install -q transformers datasets torch scikit-learn pandas numpy seqeval accelerate
# transformers: Thư viện HuggingFace (chứa PhoBERT).
# datasets: Xử lý dataset.
# torch: PyTorch (framework AI).
# scikit-learn: Tính metrics (accuracy, F1...).
# pandas: Xử lý data dạng bảng.
# seqeval: Đánh giá sequence labeling.
# accelerate: Tăng tốc training.

# Tắt wandb để tránh yêu cầu API key
import os
os.environ['WANDB_DISABLED'] = 'true'
print("✅ Đã tắt Weights & Biases tracking")

## 📁 2. Upload file data

Upload file `intent_data_augmented_abcxyz.csv` từ máy

In [ ]:
from google.colab import files
import pandas as pd

# Upload file CSV
print(" Vui lòng upload file intent_data_augmented.csv")
uploaded = files.upload()

# Lấy tên file đầu tiên được upload
data_file = list(uploaded.keys())[0]
print(f"✅ Đã upload file: {data_file}")

# Đọc và kiểm tra data
# đọc file csv thành dataframe
# text	                 |       label
# "Bật đèn khu A"	     |   device_control
# "Doanh thu tháng này"	 |   financial_query
# "Cách trồng lúa"	     |   knowledge_query
df = pd.read_csv(data_file)
print(f"\n📊 Tổng số mẫu: {len(df)}")
print(f"\n📋 Các cột: {df.columns.tolist()}")
print(f"\n🏷️ Phân bố labels:")
print(df['label'].value_counts())
print(f"\n👀 Xem 5 mẫu đầu tiên:")
df.head()

## 🔧 3. Chuẩn bị dữ liệu

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import json

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# Tạo label mapping
unique_labels = sorted(df['label'].unique()) #sort theo alphabet
# ['device_control', 'financial_query', 'knowledge_query', 'sensor_query', 'unknown']
# Model AI chỉ hiểu số, ko hiểu chữ ====> cần map chữ(intent_label) -> số(index)
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
# {
#   'device_control': 0,
#   'financial_query': 1,
#   'knowledge_query': 2,
#   'sensor_query': 3,
#   'unknown': 4
# }
id_to_label = {idx: label for label, idx in label_to_id.items()}
# {
#     "0": "device_control",
#     "1": "financial_query",
#     "2": "knowledge_query",
#     "3": "sensor_query",
#     "4": "unknown"
#   }

print(f"\n🏷️ Label mapping:")
# 🏷️ Label mapping:
#    0: device_control
#    1: financial_query
#    2: knowledge_query
#    3: sensor_query
#    4: unknown
for label, idx in label_to_id.items():
    print(f"   {idx}: {label}")

# Thêm cột label_id
df['label_id'] = df['label'].map(label_to_id)

# Chia train/val/test (70/15/15)
# train 70% dùng để học, validation 15% dùng để điều chỉnh trong quá trình học
# test 15% dùng để đánh giá cuối cùng
# lấy 30% cho dữ liệu cho phần thứ 2
# stratify Giữ tỷ lệ các label giống nhau ở cả 2 phần.
# random_state=42
# Seed cho random number generator.
# Đảm bảo mỗi lần chạy code, kết quả chia giống nhau.
# Số 42 không có ý nghĩa đặc biệt (có thể dùng số khác).
# Data gốc (1000 mẫu)
#     |
#     v
# [train_test_split] (test_size=0.3)
#     |
#     +---> train_df (700 mẫu - 70%)
#     |
#     +---> temp_df (300 mẫu - 30%)
#               |
#               v
#           [train_test_split] (test_size=0.5)
#               |
#               +---> val_df (150 mẫu - 15%)
#               |
#               +---> test_df (150 mẫu - 15%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"\n📊 Phân chia dữ liệu:")
print(f"   Train: {len(train_df)} mẫu")
print(f"   Val:   {len(val_df)} mẫu")
print(f"   Test:  {len(test_df)} mẫu")

# Load tokenizer của model
model_name = "vinai/phobert-base"
print(f"\n🔤 Loading tokenizer: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("✅ Tokenizer loaded")

## 🎯 4. Tạo Dataset

In [ ]:
from torch.utils.data import Dataset
# Dataset là 1 class đặc biệt giúp lưu trữ dlieu(texts,labels),
# cung cấp cách lấy mẫu dữ liệu
# tự động tokenize khi cần
# extends class Dataset
"""
Tạo 1 khuôn mẫu để đóng gói dữ liệu
Mỗi câu văn(text) đc chuyển thành:
 - input_ids: Chuỗi số đại diện cho câu văn
 - attention_mask: Đánh dấu phần nào là văn bản thật, phần nào là padding (đệm)
 - labels: Nhãn ý định (dạng số)
"""
class IntentDataset(Dataset):
    # lưu trữ texts, labels, tokenizer
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts #list các câu text
        self.labels = labels#list các label id
        self.tokenizer = tokenizer#chuyển chữ -> token ids
        self.max_length = max_length#limit 256 token
    # pytorch cần biết có bao nhiêu mẫu để tạo batches(chia dữ liệu thành nhóm)
    def __len__(self):
        return len(self.texts)
    # trả về số lượng mẫu trong dataset
    def __getitem__(self, idx):
        text = str(self.texts[idx])# lấy câu thứ idx
        label = self.labels[idx]#lấy label thứ idx
        
        encoding = self.tokenizer(
            text,
            truncation=True, #cắt nếu quá dài
            max_length=self.max_length,#tối đa 256 token
            padding='max_length',#thêm padding để đủ 256
            return_tensors='pt'#trả về pytorch tensor
        )
        #trả về dict chứa 
        return {
            # tokenizer trả về shape 2D
            # PyTorch Dataloader cần shape 1D
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
    """
    đc gọi khi lấy 1 mẫu dataset[0], tokenize text thành số
    trả về dict chứa 
    input_ids: Mảng ID của tokens.
    attention_mask: Mảng đánh dấu token nào là thật, token nào là padding.
    labels: Label ID (0, 1, 2...).
    """
# Tạo datasets cho train, val, test
train_dataset = IntentDataset(
    train_df['text'].tolist(),
    train_df['label_id'].tolist(),
    tokenizer
)

val_dataset = IntentDataset(
    val_df['text'].tolist(),
    val_df['label_id'].tolist(),
    tokenizer
)

test_dataset = IntentDataset(
    test_df['text'].tolist(),
    test_df['label_id'].tolist(),
    tokenizer
)

# Text (câu người dùng)
#  → Dataset
#  → DataLoader (batch)
#  → Tokenizer
#  → Model
#  → Loss / Prediction
print("✅ Datasets created")

## 🚀 5. Training Model

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np
"""
Tải model PhoBERT
num_train_epochs=5: Học lặp lại 5 lần toàn bộ dữ liệu
batch_size=16: Mỗi lần học 16 câu cùng lúc
early_stopping: Dừng sớm nếu không cải thiện sau 2 lần kiểm tra
Accuracy: Tỷ lệ dự đoán đúng
F1 Score: Điểm cân bằng giữa precision và recall
Precision: Độ chính xác khi dự đoán
Recall: Khả năng tìm ra tất cả các trường hợp đúng
"""
# Load model
print(f"🤖 Loading model: {model_name}")
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_to_id) #5 intent
)
model.to(device)
print("✅ Model loaded")

# Định nghĩa metrics
def compute_metrics(pred):
    labels = pred.label_ids #label thật
    preds = pred.predictions.argmax(-1) #dự đoán của model
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    """
    Các metrics:
    Accuracy: Tỷ lệ dự đoán đúng.
    Precision: Trong số dự đoán là X, bao nhiêu % đúng là X.
    Recall: Trong số thật là X, bao nhiêu % được bắt đúng.
    F1: Trung bình hài hòa của Precision và Recall.
    """
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./intent_classifier_output',
    num_train_epochs=5, # train 5 lần qua toàn bộ data
    per_device_train_batch_size=16,#mỗi lần train 16 mẫu
    per_device_eval_batch_size=32,#mỗi lần eval 32 mẫu
    warmup_steps=500,# 500 bước đầu tăng learning rate từ từ
    weight_decay=0.01,# tránh overfit
    logging_dir='./logs',#
    logging_steps=50,#
    eval_strategy="epoch",#eval sau mỗi epoch
    save_strategy="epoch",#lưu model sau mỗi epoch
    load_best_model_at_end=True,#lấy model tốt nhất
    metric_for_best_model="f1",#dựa vào F1 score
    greater_is_better=True,#
    save_total_limit=2,#chỉ giữ 2 checkpoint gần nhất
    fp16=torch.cuda.is_available(),  # Mixed precision nếu có GPU#
    report_to="none",  # Tắt wandb/tensorboard#
)

# Trainer
# nếu F1 ko cải thiện sau 2 epochs -> dừng training tránh overfit
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)
"""
# Trainer tự động tạo DataLoader
# DataLoader sẽ:
# 1. Gọi __getitem__(0), __getitem__(1), ..., __getitem__(15)
# 2. Gộp 16 mẫu thành 1 batch
# 3. Đưa vào model để huấn luyện
"""

"""
outputs = model(
    input_ids=batch['input_ids'],        # (16, 256)
    attention_mask=batch['attention_mask'], # (16, 256)
    labels=batch['labels']                # (16,)
)

outputs = {
    'loss': tensor(0.5234),           # Độ sai lệch tổng thể
    'logits': tensor([...]),          # Dự đoán thô cho 16 mẫu
    'hidden_states': None,            # (Optional) Trạng thái ẩn
    'attentions': None                # (Optional) Attention weights
}
"""

print("\n🏋️ Bắt đầu training...")
print("=" * 60)

# Train
trainer.train()
"""
Trong quá trình train 
Lấy 16 mẫu từ train_dataset.
Đưa qua model → Dự đoán.
Tính loss (sai số).
Backpropagation (cập nhật trọng số).
Lặp lại cho đến hết data → 1 epoch.
Sau mỗi epoch → Eval trên val_dataset.
Lặp lại 5 epochs (hoặc dừng sớm nếu không cải thiện).
"""

print("\n✅ Training hoàn thành!")

## 📊 6. Đánh giá Model

In [ ]:
# Evaluate trên test set
print("📊 Đánh giá trên Test Set:")
print("=" * 60)

test_results = trainer.evaluate(test_dataset)
print(f"\n📈 Test Results:")
for key, value in test_results.items():
    print(f"   {key}: {value:.4f}")

# Detailed classification report
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

print(f"\n📋 Classification Report:")
print("=" * 60)
print(classification_report(
    labels,
    preds,
    target_names=[id_to_label[i] for i in range(len(id_to_label))],
    digits=4
))

## 🧪 7. Test với câu mẫu

In [ ]:
def predict_intent(text, model, tokenizer, label_mapping):
    """Dự đoán intent cho một câu"""
    model.eval()
    
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding=True
    )
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
    
    # Top 3 predictions
    top_probs, top_indices = torch.topk(probs[0], k=3)
    
    print(f"\n📝 Text: {text}")
    print(f"\n🎯 Predictions:")
    for prob, idx in zip(top_probs.cpu().numpy(), top_indices.cpu().numpy()):
        intent = label_mapping[idx]
        print(f"   {intent}: {prob:.4f} ({prob*100:.2f}%)")

# Test với các câu mẫu
test_sentences = [
    "Mật độ trồng kinh giới thích hợp?",
    "Dữ liệu cảm biến 1 giờ trước đi",
    "Lên lịch máy bơm mỗi 2 tiếng.",
    "Báo cáo chi phí theo từng hạng mục.",
    "Bạn là ai?"
]

print("\n" + "=" * 60)
print("🧪 TEST VỚI CÁC CÂU MẪU")
print("=" * 60)

for sentence in test_sentences:
    predict_intent(sentence, model, tokenizer, id_to_label)
    print("-" * 60)

## 💾 8. Lưu Model

In [ ]:
import os
import shutil

# Tạo thư mục output
output_dir = "./intent_classifier_final"
os.makedirs(output_dir, exist_ok=True)

print(f"💾 Lưu model vào: {output_dir}")

# Lưu model và tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Lưu label mapping
label_mapping = {
    "label_to_id": label_to_id,
    "id_to_label": id_to_label
}

with open(os.path.join(output_dir, "label_mapping.json"), "w", encoding="utf-8") as f:
    json.dump(label_mapping, f, ensure_ascii=False, indent=2)

print("✅ Model đã được lưu!")
print(f"\n📁 Các file trong {output_dir}:")
for file in os.listdir(output_dir):
    print(f"   - {file}")

## 📥 9. Download Model về máy

In [ ]:
# Nén thư mục model thành file zip
print("📦 Đang nén model...")
shutil.make_archive("intent_classifier_final", 'zip', output_dir)
print("✅ Đã nén xong!")

# Download file zip
print("\n📥 Download file zip về máy...")
files.download("intent_classifier_final.zip")

print("\n" + "=" * 60)
print("✅ HOÀN THÀNH!")
print("=" * 60)
print("\n📝 Hướng dẫn sử dụng:")
print("   1. Giải nén file intent_classifier_final.zip")
print("   2. Copy toàn bộ nội dung vào thư mục:")
print("      apps/python-ai-service/models/intent_classifier/")
print("   3. Restart Python AI Service")
print("   4. Test API để kiểm tra model mới")
print("\n🎉 Chúc mừng! Model đã được train thành công!")